In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import requests
import yaml
import json
import os
import pandas as pd
import gzip
import shutil

In [52]:
from gdc.download import api_download_iterative

Config

In [53]:
with open('conf/user_conf.yaml', 'r') as f:
    conf = yaml.load(f, Loader=yaml.FullLoader)

Paths

In [54]:
slides_metadata_file = os.path.join(conf['data_path'], 'slides_metadata.csv')
rnaseq_metadata_file = os.path.join(conf['data_path'], 'rnaseq_metadata.csv')

In [55]:
slides_path = os.path.join(conf['data_path'], 'slides', 'svs')
if not os.path.exists(slides_path):
    os.mkdir(slides_path)

In [56]:
rnaseq_path = os.path.join(conf['data_path'], 'rnaseq')

if not os.path.exists(rnaseq_path):
    os.mkdir(rnaseq_path)

# Download Files

Read metadata

In [57]:
slides_df = pd.read_csv(slides_metadata_file, sep='|')
slides_df = slides_df[['file_name', 'file_id', 'file_size', 'sample_id', 'experimental_strategy']]
slides_df.head(3)

,file_name,file_id,file_size,sample_id,experimental_strategy
0,TCGA-2L-AAQM-11A-01-TSA.svs,ddce0ab5-47b7-43c2-8d67-3d50c48d8ae7,60.99,TCGA-2L-AAQM-11A,Tissue Slide
1,TCGA-HZ-A49I-01A-01-TS1.svs,7ec5c9d0-8c22-4e30-bae8-605c3dcfe744,293.72,TCGA-HZ-A49I-01A,Tissue Slide
2,TCGA-3A-A9IZ-01A-01-TS1.svs,265adb6b-913e-490c-8cea-32d67c1440a2,166.95,TCGA-3A-A9IZ-01A,Tissue Slide


In [58]:
rnaseq_df = pd.read_csv(rnaseq_metadata_file, sep='|')
rnaseq_df = rnaseq_df[['file_name', 'file_id', 'file_size', 'sample_id', 'workflow_type']]
rnaseq_df.head(3)

,file_name,file_id,file_size,sample_id,workflow_type
0,TCGA-FB-AAPS-01A_HTSeq-FPKM-UQ.txt.gz,1e0be26b-da47-4e56-926c-9afc259a2bd0,0.53,TCGA-FB-AAPS-01A,HTSeq - FPKM-UQ
1,TCGA-FB-AAQ3-01A_HTSeq-FPKM-UQ.txt.gz,2803abb9-bf21-4b75-a3c9-687c87b75701,0.50,TCGA-FB-AAQ3-01A,HTSeq - FPKM-UQ
2,TCGA-HZ-8005-01A_HTSeq-Counts.txt.gz,4d98a852-c30a-437b-a214-ee81015d674b,0.25,TCGA-HZ-8005-01A,HTSeq - Counts


## Slides

*Keep only the ones with RNASeq related data?*
*Keep only tissue slides?*

Number of files and count

In [59]:
summary = slides_df.groupby('experimental_strategy').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
experimental_strategy,,
Diagnostic Slide,209,244.53
Tissue Slide,257,57.48


In [60]:
rna_seq_samples = rnaseq_df['sample_id'].unique()

Keep only the ones with rnaseq data

In [63]:
# slides_df = slides_df[slides_df['sample_id'].isin(rna_seq_samples)]

Keep only Tissue Slides

In [ ]:
# slides_df = slides_df[slides_df['experimental_strategy'] == 'Tissue Slide']

In [64]:
summary = slides_df.groupby('experimental_strategy').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
experimental_strategy,,
Tissue Slide,257,57.48


In [65]:
# Download only failed
files_downloaded = os.listdir(slides_path)
slides_df = slides_df[-slides_df['file_name'].isin(files_downloaded)]

In [67]:
summary = slides_df.groupby('experimental_strategy').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
experimental_strategy,,
Tissue Slide,48,5.68


In [68]:
api_download_iterative(slides_df, slides_path, multiprocess=8)

['TCGA-2L-AAQA-11A-01-TSA.svs',
 'TCGA-2L-AAQM-11A-01-TSA.svs',
 'TCGA-FZ-5922-11A-01-TS1.svs',
 'TCGA-IB-7647-01A-01-TS1.svs',
 'TCGA-US-A774-11A-01-TSA.svs',
 'TCGA-PZ-A5RE-11A-01-TS1.svs',
 'TCGA-FZ-5922-01A-01-TS1.svs',
 'TCGA-L1-A7W4-11A-01-TSA.svs',
 'TCGA-FB-AAQ1-11A-01-TSA.svs',
 'TCGA-US-A77E-11A-01-TSA.svs',
 'TCGA-FB-AAQ0-11A-01-TSA.svs',
 'TCGA-2L-AAQE-11A-01-TS1.svs',
 'TCGA-FZ-5920-01A-01-BS1.svs',
 'TCGA-FB-AAPY-11A-01-TSA.svs',
 'TCGA-FZ-5919-01A-01-TS1.svs',
 'TCGA-FZ-5920-11A-01-TS1.svs',
 'TCGA-2L-AAQJ-11A-01-TS1.svs',
 'TCGA-FB-AAPU-11A-01-TSA.svs',
 'TCGA-US-A77G-11A-01-TSA.svs',
 'TCGA-FZ-5922-01A-01-BS1.svs',
 'TCGA-FZ-5926-01A-01-BS1.svs',
 'TCGA-FB-AAQ2-11A-01-TSA.svs',
 'TCGA-FZ-5923-11A-01-TS1.svs',
 'TCGA-FZ-5920-01A-01-TS1.svs',
 'TCGA-US-A779-11A-01-TS1.svs',
 'TCGA-FB-AAPQ-11A-01-TS1.svs',
 'TCGA-FB-AAQ6-11A-01-TSA.svs',
 'TCGA-XD-AAUH-11A-01-TSA.svs',
 'TCGA-FB-AAQ3-11A-01-TSA.svs',
 'TCGA-FZ-5919-11A-01-TS1.svs',
 'TCGA-2L-AAQI-11A-01-TS1.svs',
 'TCGA-F

## RNA-Seq

In [ ]:
summary = rnaseq_df.groupby('workflow_type').size()
summary.reset_index()

In [ ]:
summary = rnaseq_df.groupby('workflow_type').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

In [ ]:
slides_samples = slides_df['sample_id'].unique()

In [ ]:
rnaseq_df = rnaseq_df[rnaseq_df['sample_id'].isin(slides_samples)]

In [ ]:
summary = rnaseq_df.groupby('workflow_type').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

In [ ]:
files = api_download_iterative(rnaseq_df, rnaseq_path, multiprocess=8)

In [ ]:
def gunzip(source_filepath, dest_filepath):
    with gzip.open(source_filepath, 'rb') as s_file:
        with open(dest_filepath, 'wb') as d_file:
            shutil.copyfileobj(s_file, d_file)

In [ ]:
for file_name in files:
    
    source_filepath = os.path.join(rnaseq_path, file_name)
    dest_filepath = re.sub('\.gz$', '', source_filepath)
    
    gunzip(source_filepath, dest_filepath)
    os.remove(source_filepath)